## Lesson: Advanced Jupyter Notebook
Supporting notebook for the lesson based on [this blog](https://blog.esciencecenter.nl/easy-ipcc-powered-by-esmvalcore-19a0b6366ea7).

### Dataset
- object class used to define and find datasets
- see API [reference](https://docs.esmvaltool.org/projects/ESMValCore/en/latest/api/esmvalcore.dataset.html#esmvalcore.dataset.Dataset)
  

In [ ]:
from esmvalcore.dataset import Dataset

dataset = Dataset(
    short_name='tos',
    mip='Omon',
    project='CMIP6',
    exp='historical',
    dataset='ACCESS-ESM1-5',
    ensemble='r4i1p1f1',
    grid='gn',
)
dataset.augment_facets()
print(dataset)

## Local and ESGF search
- ESMValTool can find datasets locally with root paths defined in your user config
- ESMValTool can also search on ESGF nodes when it can access the internet
- Use `'*'` for a wildcard search

In [ ]:
from esmvalcore.config import CFG

# CFG['search_esgf'] = 'always'
dataset_search = Dataset(
    short_name='tos',
    mip='Omon',
    project='CMIP6',
    exp='historical',
    dataset='ACCESS-ESM1-5',
    ensemble='*',
    grid='gn',
)
ensemble_datasets = list(dataset_search.from_files())

print([ds['ensemble'] for ds in ensemble_datasets])

In [ ]:
# [dataset.files[0] for dataset in ensemble_datasets]

### Supplementary variables

In [ ]:
# Discard augmented facets as they will be different for areacello
dataset = Dataset(**dataset.minimal_facets)

# Add areacello as supplementary dataset
dataset.add_supplementary(short_name='areacello', mip='Ofx')

# Autocomplete and inspect
dataset.augment_facets()
print(dataset.summary())

## Load data

In [ ]:
# Before load
print(dataset.files)

cube = dataset.load()
cube

## Preprocessors
- refer to [documentation](https://docs.esmvaltool.org/projects/esmvalcore/en/latest/recipe/overview.html#recipe-section-preprocessors) on what preprocessors are
- refer to [API reference](https://docs.esmvaltool.org/projects/ESMValCore/en/latest/api/esmvalcore.preprocessor.html) on using specific preprocessors

In [ ]:
from esmvalcore.preprocessor import annual_statistics, anomalies, area_statistics

# Set the reference period for anomalies 
reference_period = {
    "start_year": 1950, "start_month": 1, "start_day": 1,
    "end_year": 1979, "end_month": 12, "end_day": 31,
}

cube = area_statistics(cube, operator='mean')
cube = anomalies(cube, reference=reference_period, period='month')
cube = annual_statistics(cube, operator='mean')
cube.convert_units('degrees_C')
cube

### Custom code
- Use this space to add your own analysis code to this preprocessed data

In [ ]:
import xarray as xr
da = xr.DataArray.from_iris(cube)
da.plot()
print(da)

## Apply workflow to multiple datasets
- use `Dataset` object to make a list of datasets
- apply the same preprocessors and diagnostic to each

In [ ]:
import cf_units
import matplotlib.pyplot as plt
from iris import quickplot

from esmvalcore.config import CFG
from esmvalcore.dataset import Dataset
from esmvalcore.preprocessor import annual_statistics, anomalies, area_statistics


# Settings for automatic ESGF search
CFG['search_esgf'] = 'when_missing'

# Declare common dataset facets
template = Dataset(
    short_name='tos',
    mip='Omon',
    project='CMIP6',
    exp= '*', # We'll fill this below
    dataset='*',  # We'll fill this below
    ensemble='r4i1p1f1',
    grid='gn',
    timerange='1850/2100'  #ACCESS-ESM1-5 extends to 2300
)

# Substitute data sources and experiments
datasets = []
for dataset_id in ["CESM2", "MPI-ESM1-2-LR", "ACCESS-ESM1-5"]:
    for experiment_id in ['ssp126', 'ssp585']:
        dataset = template.copy(dataset=dataset_id, exp=['historical', experiment_id])
        dataset.add_supplementary(short_name='areacello', mip='Ofx', exp='historical')
        dataset.augment_facets()
        datasets.append(dataset)

# Set the reference period for anomalies 
reference_period = {
    "start_year": 1950, "start_month": 1, "start_day": 1,
    "end_year": 1979, "end_month": 12, "end_day": 31,
}

# (Down)load, pre-process, and plot the cubes
for dataset in datasets: 
    cube = dataset.load()
    cube = area_statistics(cube, operator='mean')
    cube = anomalies(cube, reference=reference_period, period='month')  # notice 'month'
    cube = annual_statistics(cube, operator='mean')
    cube.convert_units('degrees_C')

    # Make sure all datasets use the same calendar for plotting
    tcoord = cube.coord('time')
    tcoord.units = cf_units.Unit(tcoord.units.origin, calendar='gregorian')

    # Plot
    quickplot.plot(cube, label=f"{dataset['dataset']} - {dataset['exp']}")

# Show the plot
plt.legend()
# plt.savefig('./esmValTool/figure.png')
plt.show()

## Helper to start making a recipe
- can help start to write a recipe for the selected datasets
- you will have to add in preprocessor and diagnostic sections
- an example recipe of this kind of diagnostic is `examples/recipe_easy_ipcc.yml`you can find in your cloned ESMValTool repo or can be found in available recipes

In [ ]:
from esmvalcore.dataset import datasets_to_recipe
import yaml

# these are the datasets from above
for dataset in datasets:
    dataset.facets['diagnostic'] = 'easy_ipcc'
print(yaml.safe_dump(datasets_to_recipe(datasets)))

In [ ]:
## find more datasets

template = Dataset(
    short_name='tos',
    mip='Omon',
    activity='CMIP',
    institute='*', # facet req. to search locally
    project='CMIP6',
    exp= ['historical', 'ssp585'], #,'ssp126'
    dataset='*',  #
    ensemble='*',
    grid='gn',
    timerange='1850/2100'  
)

all_datasets = list(template.from_files())
for dataset in all_datasets:
    dataset.facets['diagnostic'] = 'easy_ipcc'

print(len(all_datasets))
print(yaml.safe_dump(datasets_to_recipe(all_datasets)))